### Multi-Class Classification
Now that we've explored a binary classification problem with linear and non-linear architectures, we now want to shift to a multi-class problem where there are more than two options that the model needs to be able to classify.

The multi-class data will be artificial data from the scikit-learn `make_blobs()` function. The general flow is as follows:
1. Make the artificial data and convert to tensors
2. Visualize the data
3. Define the model architecture
4. Train the model
5. Adjust hyperparameters as necessary